先用基础paddle api实现一遍波士顿房价预测代码

In [1]:
#加载飞桨、NumPy和相关类库
import paddle
from paddle.nn import Linear
import paddle.nn.functional as F
import numpy as np
import os
import random

In [4]:
def load_data():
    # 从文件导入数据
    datafile = './dataset/housing.data'
    data = np.fromfile(datafile, sep=' ', dtype=np.float32)

    # 每条数据包括14项，其中前面13项是影响因素，第14项是相应的房屋价格中位数
    feature_names = [ 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', \
                      'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV' ]
    feature_num = len(feature_names)

    # 将原始数据进行Reshape，变成[N, 14]这样的形状
    data = data.reshape([data.shape[0] // feature_num, feature_num])

    # 将原数据集拆分成训练集和测试集
    # 这里使用80%的数据做训练，20%的数据做测试
    # 测试集和训练集必须是没有交集的
    ratio = 0.8
    offset = int(data.shape[0] * ratio)
    training_data = data[:offset]

    # 计算train数据集的最大值，最小值
    maximums, minimums = training_data.max(axis=0), training_data.min(axis=0)
    
    # 记录数据的归一化参数，在预测时对数据做归一化
    global max_values
    global min_values
    
    max_values = maximums
    min_values = minimums
    
    # 对数据进行归一化处理
    for i in range(feature_num):
        data[:, i] = (data[:, i] - min_values[i]) / (maximums[i] - minimums[i])

    # 训练集和测试集的划分比例
    training_data = data[:offset]
    test_data = data[offset:]
    return training_data, test_data

In [5]:
# 验证数据集读取程序的正确性
training_data, test_data = load_data()
print(training_data.shape)
print(training_data[1,:])

(404, 14)
[2.35922547e-04 0.00000000e+00 2.62405723e-01 0.00000000e+00
 1.72839552e-01 5.47997713e-01 7.82698274e-01 3.48961979e-01
 4.34782617e-02 1.14822544e-01 5.53191364e-01 1.00000000e+00
 2.04470202e-01 3.68888885e-01]


In [6]:

class Regressor(paddle.nn.Layer):

    # self代表类的实例自身
    def __init__(self):
        # 初始化父类中的一些参数
        super(Regressor, self).__init__()
        
        # 定义一层全连接层，输入维度是13，输出维度是1
        self.fc = Linear(in_features=13, out_features=1)
    
    # 网络的前向计算
    def forward(self, inputs):
        x = self.fc(inputs)
        return x

In [7]:
# 声明定义好的线性回归模型
model = Regressor()
# 开启模型训练模式,其实就是开启梯队计算
model.train()
# 加载数据
training_data, test_data = load_data()
# 定义优化算法，使用随机梯度下降SGD
# 学习率设置为0.01
opt = paddle.optimizer.SGD(learning_rate=0.01, parameters=model.parameters())

In [8]:
EPOCH_NUM = 10   # 设置外层循环次数
BATCH_SIZE = 10  # 设置batch大小

# 定义外层循环
for epoch_id in range(EPOCH_NUM):
    # 在每轮迭代开始之前，将训练数据的顺序随机的打乱
    np.random.shuffle(training_data)
    # 将训练数据进行拆分，每个batch包含10条数据
    mini_batches = [training_data[k:k+BATCH_SIZE] for k in range(0, len(training_data), BATCH_SIZE)]
    # 定义内层循环
    for iter_id, mini_batch in enumerate(mini_batches):
        x = np.array(mini_batch[:, :-1]) # 获得当前批次训练数据
        y = np.array(mini_batch[:, -1:]) # 获得当前批次训练标签（真实房价）
        # 将numpy数据转为飞桨动态图tensor的格式
        house_features = paddle.to_tensor(x,dtype='float32')
        prices = paddle.to_tensor(y,dtype='float32')
        
        # 前向计算
        predicts = model(house_features)

        # 计算损失
        loss = F.square_error_cost(predicts, label=prices)
        avg_loss = paddle.mean(loss)
        if iter_id%20==0:
            print("epoch: {}, iter: {}, loss is: {}".format(epoch_id, iter_id, avg_loss.numpy()))
        
        # 反向传播，计算每层参数的梯度值
        avg_loss.backward()
        # 更新参数，根据设置好的学习率迭代一步
        opt.step()
        # 清空梯度变量，以备下一轮计算
        opt.clear_grad()

epoch: 0, iter: 0, loss is: [0.263274]
epoch: 0, iter: 20, loss is: [0.0582662]
epoch: 0, iter: 40, loss is: [0.2719737]
epoch: 1, iter: 0, loss is: [0.03805635]
epoch: 1, iter: 20, loss is: [0.0310575]
epoch: 1, iter: 40, loss is: [0.02874544]
epoch: 2, iter: 0, loss is: [0.06718808]
epoch: 2, iter: 20, loss is: [0.08902705]
epoch: 2, iter: 40, loss is: [0.03419156]
epoch: 3, iter: 0, loss is: [0.04281816]
epoch: 3, iter: 20, loss is: [0.02486593]
epoch: 3, iter: 40, loss is: [0.07300695]
epoch: 4, iter: 0, loss is: [0.05793606]
epoch: 4, iter: 20, loss is: [0.06530922]
epoch: 4, iter: 40, loss is: [0.06916802]
epoch: 5, iter: 0, loss is: [0.0393042]
epoch: 5, iter: 20, loss is: [0.05897485]
epoch: 5, iter: 40, loss is: [0.0242769]
epoch: 6, iter: 0, loss is: [0.02160133]
epoch: 6, iter: 20, loss is: [0.0810772]
epoch: 6, iter: 40, loss is: [0.02146257]
epoch: 7, iter: 0, loss is: [0.03945627]
epoch: 7, iter: 20, loss is: [0.08160972]
epoch: 7, iter: 40, loss is: [0.0131107]
epoch: 8,

In [9]:
# 保存模型参数，文件名为LR_model.pdparams
paddle.save(model.state_dict(), './model/LR_model.pdparams')
print("模型保存成功,模型参数保存在LR_model.pdparams中")

模型保存成功,模型参数保存在LR_model.pdparams中


In [10]:
def load_one_example():
    # 从上边已加载的测试集中，随机选择一条作为测试数据
    idx = np.random.randint(0, test_data.shape[0])
    idx = -10
    one_data, label = test_data[idx, :-1], test_data[idx, -1]
    # 修改该条数据shape为[1,13]
    one_data =  one_data.reshape([1,-1])

    return one_data, label

In [13]:
# 参数为保存模型参数的文件地址
model_dict = paddle.load('./model/LR_model.pdparams')
model.load_dict(model_dict)
model.eval()


# 参数为数据集的文件地址
one_data, label = load_one_example()
# 将数据转为动态图的variable格式 
one_data = paddle.to_tensor(one_data,dtype="float32")
predict = model(one_data)

# 对结果做反归一化处理
predict = predict * (max_values[-1] - min_values[-1]) + min_values[-1]
# 对label数据做反归一化处理
label = label * (max_values[-1] - min_values[-1]) + min_values[-1]

print("Inference result is {}, the corresponding label is {}".format(predict.numpy(), label))

Inference result is [[18.295418]], the corresponding label is 19.700000762939453


接下来，使用paddle高级api实现一遍，更加的精简


In [14]:
import paddle
paddle.set_default_dtype("float32")

# 用高层API定义数据集，无需进行数据处理等，高层API为你一条龙搞定
train_dataset = paddle.text.datasets.UCIHousing(mode='train')
eval_dataset = paddle.text.datasets.UCIHousing(mode='test')

# 训练模型
model = paddle.Model(Regressor())
model.prepare(optimizer=paddle.optimizer.SGD(learning_rate=0.005, parameters=model.parameters()),
              loss=paddle.nn.MSELoss())
model.fit(train_dataset, eval_dataset, epochs=10, batch_size=8, verbose=1)

result = model.evaluate(eval_dataset, batch_size=8)
print("result:",result)

result_pred = model.predict(one_data, batch_size=1) # result_pred是一个list，元素数目对应模型的输出数目
result_pred = result_pred[0] # tuple,其中第一个值是个array
print("Inference result is {}, the corresponding label is {}".format(result_pred[0][0], label))

Cache file C:\Users\shimon\.cache\paddle\dataset\uci_housing\housing.data not found, downloading http://paddlemodels.bj.bcebos.com/uci_housing/housing.data 
Begin to download


item 12/12 [==========================>...] - ETA: 0s - 15ms/itemThe loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/10
step 51/51 [==============================] - loss: 228.0802 - 1ms/step            
Eval begin...
step 13/13 [==============================] - loss: 104.0797 - 726us/step          
Eval samples: 102
Epoch 2/10
step 51/51 [==============================] - loss: 27.5447 - 901us/step           
Eval begin...
step 13/13 [==============================] - loss: 29.2368 - 712us/step          
Eval samples: 102
Epoch 3/10
step 50/51 [============================>.] - loss: 168.1972 - ETA: 0s - 917us/step


Download finished


step 51/51 [==============================] - loss: 13.0833 - 919us/step           
Eval begin...
step 13/13 [==============================] - loss: 15.4721 - 699us/step          
Eval samples: 102
Epoch 4/10
step 51/51 [==============================] - loss: 18.0030 - 993us/step          
Eval begin...
step 13/13 [==============================] - loss: 17.1943 - 843us/step          
Eval samples: 102
Epoch 5/10
step 51/51 [==============================] - loss: 11.5348 - 955us/step          
Eval begin...
step 13/13 [==============================] - loss: 21.1768 - 692us/step          
Eval samples: 102
Epoch 6/10
step 51/51 [==============================] - loss: 24.3541 - 860us/step          
Eval begin...
step 13/13 [==============================] - loss: 24.7034 - 602us/step          
Eval samples: 102
Epoch 7/10
step 51/51 [==============================] - loss: 19.4659 - 878us/step          
Eval begin...
step 13/13 [==============================] - loss: 26.5972 - 564u